# Setup

In [ ]:
%pip install magtrack matplotlib

In [ ]:
import magtrack
from magtrack.simulation import simulate_beads
import numpy as np
import cupy as cp
from time import perf_counter
import matplotlib.pyplot as plt

## Simulation
To start we will simulate a stack of 100 images of beads. In each frame the bead will drift slightly to the right with a sin motion up and down.

In [ ]:
# Set up parameters
n_frames = 100
roi = 64
nm_per_px = 100.0

# Create simulation positions
x_true = np.linspace(-200, 200, n_frames)
y_true = 100. * np.sin(np.linspace(1, 6*np.pi, n_frames))
z_true = np.zeros_like(x_true)
xyz_true = np.stack([x_true, y_true, z_true], axis=1)

# Simulate stack of beads
stack = simulate_beads(xyz_true, size_px=roi, nm_per_px=nm_per_px)

# Simulate bead positions from the corner
xy_offset = roi * nm_per_px / 2
x_true_nm = x_true + xy_offset
y_true_nm = y_true + xy_offset
x_true_px = x_true_nm / nm_per_px
y_true_px = y_true_nm / nm_per_px

# Create the plots
plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
plt.imshow(stack[:, :, 0], cmap='gray', origin='lower')
plt.title('First Frame')

plt.subplot(1, 3, 2)
plt.imshow(stack[:, :, -1], cmap='gray', origin='lower')
plt.title('Last Frame')

plt.subplot(1, 3, 3)
plt.plot(x_true_nm, y_true_nm, marker='o')
plt.title('XY Position')
plt.ylabel('y (nm)')
plt.xlabel('x (nm)')

plt.tight_layout()
plt.show()

# Example 1: Get XY with Center-of-Mass
We will now try getting a rough xy position with the center_of_mass algorithm. The center-of-mass algorithm is easily biased by the images' background. A key-word argument `background` can help with this. By default `background='none'` which does nothing to the background. This is faster than the other two options but less accurate. The other two options are `background='mean'` or `background='median'` which calculate the mean or median of each image in the back and subtrack it from the repective image. `background='median'` is often the slowest but most accurate option.

In [ ]:
# Estimate xy with center-of-mass
x_com_px, y_com_px = magtrack.center_of_mass(stack, background="median")

# Convert from pixels to nm
x_com_nm = x_com_px * nm_per_px
y_com_nm = y_com_px * nm_per_px

# Calculate error
dx = x_com_nm - x_true_nm
dy = y_com_nm - y_true_nm
error_nm = np.sqrt(dx**2 + dy**2)

# Plots
plt.figure(figsize=(6, 6))

plt.subplot(2, 2, 1)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_com_px[0], y_com_px[0], 'rx', label='Center-of-Mass')
plt.xlim(0, roi)
plt.ylim(0, roi)
plt.legend()
plt.title('First Frame')

plt.subplot(2, 2, 2)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_com_px[0], y_com_px[0], 'rx', label='Center-of-Mass')
plt.xlim(x_true_px[0]-5, x_true_px[0]+5)
plt.ylim(y_true_px[0]-5, y_true_px[0]+5)
plt.legend()
plt.title('First Frame (Zoom)')

plt.subplot(2, 2, 3)
plt.plot(x_true_nm, y_true_nm, label='True')
plt.plot(x_com_nm, y_com_nm, label='Center-of-Mass')
plt.legend()
plt.title('XY Position')
plt.ylabel('y (nm)')
plt.xlabel('x (nm)')

plt.subplot(2, 2, 4)
plt.plot(error_nm, label='Error')
plt.legend()
plt.title('Error')
plt.ylabel('error (nm)')
plt.xlabel('Frame #')

plt.tight_layout()
plt.show()

# Example 2: Refine the XY using sub-pixel auto convolution
There are several options to refine the xy position. One way is to use auto-convolution. Auto-convolution requires you to first have an estimate of where the true center it. It then performs a auto-convolution on the row of pixels in that center to find x and a column of pixel to find y. There are four auto-convolution methods: `magtrack.auto_conv`, `magtrack.auto_conv_sub_pixel`, `magtrack.auto_conv_multiline`, or `magtrack.auto_conv_multiline_para_fit`. Two use multiple rows/columns of pixels to get more information about where the true center is. Two use sub-pixel fitting to get a much more precise estimate. Using multiple rows/column and/or performing sub-pixel fitting takes more time but is much slower. Below we will compare two of these methods to the center-of-mass.

In [ ]:
# Use the center-of-mass to get an estimate of xy
x_com_px, y_com_px = magtrack.center_of_mass(stack, background="median")

# Get a better estimate with auto-convolution
x_ac_px, y_ac_px = magtrack.auto_conv(stack, x_com_px, y_com_px)

# Get an even better estimate with the sub-pixel auto-convolution
x_acsp_px, y_acsp_px = magtrack.auto_conv_sub_pixel(stack, x_com_px, y_com_px)

# Convert from pixel to nm
x_com_nm = x_com_px * nm_per_px
y_com_nm = y_com_px * nm_per_px
x_ac_nm = x_ac_px * nm_per_px
y_ac_nm = y_ac_px * nm_per_px
x_acsp_nm = x_acsp_px * nm_per_px
y_acsp_nm = y_acsp_px * nm_per_px

# Calculate error
error_com_nm = np.sqrt((x_com_nm - x_true_nm)**2 + (y_com_nm - y_true_nm)**2)
error_ac_nm = np.sqrt((x_ac_nm - x_true_nm)**2 + (y_ac_nm - y_true_nm)**2)
error_acsp_nm = np.sqrt((x_acsp_nm - x_true_nm)**2 + (y_acsp_nm - y_true_nm)**2)

# Plots
plt.figure(figsize=(6, 12))

plt.subplot(4, 2, 1)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_com_px[0], y_com_px[0], 'rx', label='Center-of-Mass')
plt.plot(x_ac_px[0], y_ac_px[0], 'gv', label='Auto-convolution')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'm*', label='Sub-pixel')
plt.xlim(0, roi)
plt.ylim(0, roi)
plt.legend()
plt.title('First Frame')

plt.subplot(4, 2, 2)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_com_px[0], y_com_px[0], 'rx', label='Center-of-Mass')
plt.plot(x_ac_px[0], y_ac_px[0], 'gv', label='Auto-convolution')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'm*', label='Sub-pixel')
plt.xlim(x_true_px[0]-5, x_true_px[0]+5)
plt.ylim(y_true_px[0]-5, y_true_px[0]+5)
plt.title('First Frame (Zoom)')

plt.subplot(4, 2, 3)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_com_px[0], y_com_px[0], 'rx', label='Center-of-Mass')
plt.plot(x_ac_px[0], y_ac_px[0], 'gv', label='Auto-convolution')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'm*', label='Sub-pixel')
plt.xlim(x_true_px[0]-1, x_true_px[0]+1)
plt.ylim(y_true_px[0]-1, y_true_px[0]+1)
plt.title('First Frame (More Zoom)')

plt.subplot(4, 2, 4)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_com_px[0], y_com_px[0], 'rx', label='Center-of-Mass')
plt.plot(x_ac_px[0], y_ac_px[0], 'gv', label='Auto-convolution')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'm*', label='Sub-pixel')
plt.xlim(x_true_px[0]-.2, x_true_px[0]+.2)
plt.ylim(y_true_px[0]-.2, y_true_px[0]+.2)
plt.title('First Frame (Even More Zoom)')

plt.subplot(4, 2, 5)
plt.plot(x_true_nm, y_true_nm, label='True')
plt.plot(x_com_nm, y_com_nm, label='Center-of-Mass')
plt.plot(x_ac_nm, y_ac_nm, label='Auto-convolution')
plt.plot(x_acsp_nm, y_acsp_nm, label='Sub-pixel')
plt.legend()
plt.title('XY Position')
plt.ylabel('y (nm)')
plt.xlabel('x (nm)')

plt.subplot(4, 2, 6)
plt.plot(error_com_nm, label='Center-of-Mass')
plt.plot(error_ac_nm, label='Auto-convolution')
plt.plot(error_acsp_nm, label='Sub-pixel')
plt.legend()
plt.title('Error')
plt.ylabel('error (nm)')
plt.xlabel('Frame #')

plt.tight_layout()
plt.show()

# Example 3: Creating radial profiles
The `magtrack.radial_profile` function constructs radial profiles from a stack of images by averaging the light intensity at equal distances from a given center point.


In [ ]:
# Calculate the radial profiles
profiles = magtrack.radial_profile(stack, x_true_px, y_true_px)

# Plots
plt.figure(figsize=(6, 3))

plt.subplot(1, 2, 1)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True Center')
plt.plot(x_true_px[0] + np.arange(profiles.shape[0]), y_true_px[0] + (profiles[:, 0]-profiles[:, 0].mean()) * roi/4, 'r', label='Profile')
plt.xlim(0, roi)
plt.ylim(0, roi)
plt.legend()
plt.title('First Frame')

plt.subplot(1, 2, 2)
plt.plot(profiles[:, 0], 'r')
plt.ylim(0, 1)
plt.ylabel('Intensity')
plt.title('First Profile')

plt.tight_layout()
plt.show()

# Example 4: Build a Z-LUT for axial fitting
For this example we will create a new simulation that will simulate a recording of a ZLUT. We will then calculate the radial profile of each frame and use this to construct a ZLUT.


In [ ]:
# Simulate a reference stack that scans the bead in z
z_reference = np.arange(-10000, 10100, 100)  # nanometers
xyz_reference = np.column_stack([
    np.zeros_like(z_reference),
    np.zeros_like(z_reference),
    z_reference,
])
reference_stack = simulate_beads(xyz_reference, size_px=roi, nm_per_px=nm_per_px)

# Convert each frame into a radial profile centered on the bead
reference_profiles = magtrack.radial_profile(
    reference_stack,
    np.full(z_reference.shape, roi / 2, dtype=float),
    np.full(z_reference.shape, roi / 2, dtype=float)
)

# Assemble the Z-LUT: the first row stores the z positions,
# the remaining rows contain the radial profiles
zlut = np.vstack([z_reference, reference_profiles])

# Plot the ZLUT as an image
plt.figure(figsize=(6, 4))
plt.imshow(reference_profiles.T, cmap='gray', vmin=0, vmax=1, aspect=0.75)
plt.ylabel('Reference z (nm)')
plt.xlabel('Radius (pixels)')
plt.title('Z-LUT')
y_ticks = np.linspace(0, reference_profiles.shape[1]-1, 5).astype(int)
plt.yticks(y_ticks)
plt.gca().set_yticklabels(z_reference[y_ticks])
plt.tight_layout()
plt.show()

## Example 5: Calculating Z with `lookup_z`
With the Z-LUT prepared, we can analyze a fresh stack, extract its radial profiles,
and estimate the bead's axial position. The lookup function returns a sub-frame
interpolation of the best matching reference profiles.


In [ ]:
# Create a new stack with a different z trajectory
z_true_eval = np.linspace(-3000, 3000, 100) + 1000.0 * np.sin(np.linspace(0, 20 * np.pi, 100))
xyz_eval = np.column_stack([
    np.zeros_like(z_true_eval),
    np.zeros_like(z_true_eval),
    z_true_eval,
])
eval_stack = simulate_beads(xyz_eval, size_px=roi, nm_per_px=nm_per_px)

eval_profiles = magtrack.radial_profile(
    eval_stack,
    np.full(z_true_eval.shape, roi / 2, dtype=float),
    np.full(z_true_eval.shape, roi / 2, dtype=float),
)
z_fit = magtrack.lookup_z(eval_profiles, zlut)

plt.figure(figsize=(6, 6))

plt.subplot(2, 1, 1)
plt.plot(z_true_eval, label='True', linewidth=2)
plt.plot(z_fit, label='Fit', linestyle='--')
plt.xlabel('Frame number')
plt.ylabel('Z (nm)')
plt.title('Z fit from lookup_z')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(z_fit - z_true_eval, 'r-', label='Error')
plt.xlabel('Frame number')
plt.ylabel('Error (nm)')
plt.legend()

plt.tight_layout()
plt.show()


## Example 6: Refining XY with QI


In [ ]:
# Use the center-of-mass to get an estimate of xy
x_com_px, y_com_px = magtrack.center_of_mass(stack, background="median")

# Get a better estimate with the sub-pixel auto-convolution
x_acsp_px, y_acsp_px = magtrack.auto_conv_sub_pixel(stack, x_com_px, y_com_px)

# Get a better estimate with the sub-pixel QI
x_qi1_px, y_qi1_px = magtrack.qi(stack, x_com_px, y_com_px)

# Get an even better estimate by repeating sub-pixel QI
x_qi2_px, y_qi2_px = magtrack.qi(stack, x_qi1_px, y_qi1_px)
x_qi3_px, y_qi3_px = magtrack.qi(stack, x_qi2_px, y_qi2_px)

# Convert from pixel to nm
x_com_nm = x_com_px * nm_per_px
y_com_nm = y_com_px * nm_per_px
x_acsp_nm = x_acsp_px * nm_per_px
y_acsp_nm = y_acsp_px * nm_per_px
x_qi1_nm = x_qi1_px * nm_per_px
y_qi1_nm = y_qi1_px * nm_per_px
x_qi2_nm = x_qi2_px * nm_per_px
y_qi2_nm = y_qi2_px * nm_per_px
x_qi3_nm = x_qi3_px * nm_per_px
y_qi3_nm = y_qi3_px * nm_per_px

# Calculate error
error_com_nm = np.sqrt((x_com_nm - x_true_nm)**2 + (y_com_nm - y_true_nm)**2)
error_acsp_nm = np.sqrt((x_acsp_nm - x_true_nm)**2 + (y_acsp_nm - y_true_nm)**2)
error_qi1_nm = np.sqrt((x_qi1_nm - x_true_nm)**2 + (y_qi1_nm - y_true_nm)**2)
error_qi2_nm = np.sqrt((x_qi2_nm - x_true_nm)**2 + (y_qi2_nm - y_true_nm)**2)
error_qi3_nm = np.sqrt((x_qi3_nm - x_true_nm)**2 + (y_qi3_nm - y_true_nm)**2)

# Plots
plt.figure(figsize=(6, 12))

plt.subplot(4, 2, 1)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'cx', label='Auto-convolution')
plt.plot(x_qi1_px[0], y_qi1_px[0], 'g*', label='QI 1')
plt.plot(x_qi2_px[0], y_qi2_px[0], 'm*', label='QI 2')
plt.plot(x_qi3_px[0], y_qi3_px[0], 'r*', label='QI 3')
plt.xlim(0, roi)
plt.ylim(0, roi)
plt.legend()
plt.title('First Frame')

plt.subplot(4, 2, 2)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'cx', label='Auto-convolution')
plt.plot(x_qi1_px[0], y_qi1_px[0], 'g*', label='QI 1')
plt.plot(x_qi2_px[0], y_qi2_px[0], 'm*', label='QI 2')
plt.plot(x_qi3_px[0], y_qi3_px[0], 'r*', label='QI 3')
plt.xlim(x_true_px[0]-5, x_true_px[0]+5)
plt.ylim(y_true_px[0]-5, y_true_px[0]+5)
plt.title('First Frame (Zoom)')

plt.subplot(4, 2, 3)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'cx', label='Auto-convolution')
plt.plot(x_qi1_px[0], y_qi1_px[0], 'g*', label='QI 1')
plt.plot(x_qi2_px[0], y_qi2_px[0], 'm*', label='QI 2')
plt.plot(x_qi3_px[0], y_qi3_px[0], 'r*', label='QI 3')
plt.xlim(x_true_px[0]-1, x_true_px[0]+1)
plt.ylim(y_true_px[0]-1, y_true_px[0]+1)
plt.title('First Frame (More Zoom)')

plt.subplot(4, 2, 4)
plt.imshow(stack[:, :, 0], cmap='gray')
plt.plot(x_true_px[0], y_true_px[0], 'b+', label='True')
plt.plot(x_acsp_px[0], y_acsp_px[0], 'cx', label='Auto-convolution')
plt.plot(x_qi1_px[0], y_qi1_px[0], 'g*', label='QI 1')
plt.plot(x_qi2_px[0], y_qi2_px[0], 'm*', label='QI 2')
plt.plot(x_qi3_px[0], y_qi3_px[0], 'r*', label='QI 3')
plt.xlim(x_true_px[0]-.2, x_true_px[0]+.2)
plt.ylim(y_true_px[0]-.2, y_true_px[0]+.2)
plt.title('First Frame (Even More Zoom)')

plt.subplot(4, 2, 5)
plt.plot(x_true_nm, y_true_nm, label='True')
plt.plot(x_acsp_nm, y_acsp_nm, label='Auto-convolution')
plt.plot(x_qi1_nm, y_qi1_nm, label='QI 1')
plt.plot(x_qi2_nm, y_qi2_nm, label='QI 2')
plt.plot(x_qi3_nm, y_qi3_nm, label='QI 3')
plt.legend()
plt.title('XY Position')
plt.ylabel('y (nm)')
plt.xlabel('x (nm)')

plt.subplot(4, 2, 6)
plt.plot(error_com_nm, label='Center-of-Mass')
plt.plot(error_acsp_nm, label='Auto-convolution')
plt.plot(error_qi1_nm, label='QI 1')
plt.plot(error_qi2_nm, label='QI 2')
plt.plot(error_qi3_nm, label='QI 3')
plt.legend()
plt.title('Error')
plt.ylabel('error (nm)')
plt.xlabel('Frame #')

plt.tight_layout()
plt.show()

# Example 7: Using the GPU
Next we will calculate the center-of-mass on the CPU and GPU. Note to do this you must have a NVIDIA GPU. If you are running this in collab you will need to change the runtime to GPU. The block below can check if a GPU is connected. Any MagTrack function can use the CPU or GPU. If the stack of images (or other values) are on the CPU then it uses the CPU, if they are on the GPU then it uses the GPU. So before using the GPU move the stack to the GPU and then afterwords move the results back to the CPU.

In [ ]:
# Verify a GPU exists
try:
  cp.cuda.runtime.getDeviceCount()
except Exception:
  print('Error! '*10 + '\n')
  print("No GPU runtime.\n")
  print("In Colab: Runtime → Change runtime type → GPU. \n\n")
else:
  dev = cp.cuda.runtime.getDevice()
  props = cp.cuda.runtime.getDeviceProperties(dev)
  name = props["name"].decode() if isinstance(props["name"], (bytes, bytearray)) else props["name"]
  print(f"GPU: {name} | CC {props['major']}.{props['minor']} | {props['totalGlobalMem']/1e9:.1f} GB")

In [ ]:
# CPU
trials = 1000

# Warmup
for _ in range(10):
    x_com_px, y_com_px = magtrack.center_of_mass(stack, background="median")

# Real trials
start_time = perf_counter()
for _ in range(trials):
    x_com_px, y_com_px = magtrack.center_of_mass(stack, background="median")
end_time = perf_counter()

cpu_time = (end_time - start_time) / trials
print(f"CPU took an average of: {cpu_time:.6f} seconds out of {trials} trials")

In [ ]:
# GPU
trials = 1000

# Move stack to GPU
stack_gpu = cp.asarray(stack)

# Warmup
for _ in range(10):
    x_com_px, y_com_px = magtrack.center_of_mass(stack_gpu, background="median")

# Real trials
start_time = perf_counter()
for _ in range(trials):
    x_com_px, y_com_px = magtrack.center_of_mass(stack_gpu, background="median")
end_time = perf_counter()

# Move values back to CPU
x_com_px = cp.asnumpy(x_com_px)
y_com_px = cp.asnumpy(y_com_px)

gpu_time = (end_time - start_time) / trials
print(f"GPU took an average of: {gpu_time:.6f} seconds out of {trials} trials")